In [ ]:
import numpy as np
import math
# Objective function to optimize (example: Sphere function)
def objective_function(x):
    return np.sum(x**2)

# Lévy Flight distribution
def levy_flight(beta=1.5, size=1):
    sigma_u = (math.gamma(1 + beta) * np.sin(np.pi * beta / 2) /
               math.gamma((1 + beta) / 2) * beta * (2 ** ((beta - 1) / 2)))**(1 / beta)
    u = np.random.normal(0, sigma_u, size)
    v = np.random.normal(0, 1, size)
    step = u / (np.abs(v) ** (1 / beta))
    return step

# Cuckoo Search Algorithm
def cuckoo_search(objective_function, dim, lower_bound, upper_bound, num_nests=25, max_iter=100, pa=0.25):
    # Initialize nests with random solutions within bounds
    nests = np.random.rand(num_nests, dim) * (upper_bound - lower_bound) + lower_bound
    fitness = np.apply_along_axis(objective_function, 1, nests)

    # Initialize the best solution
    best_nest_idx = np.argmin(fitness)
    best_nest = nests[best_nest_idx]
    best_fitness = fitness[best_nest_idx]

    # Iterate for a fixed number of generations or until convergence
    for iteration in range(max_iter):
        for i in range(num_nests):
            # Generate a new solution using Lévy flight
            step = levy_flight(size=dim)
            new_nest = nests[i] + 0.01 * step
            new_nest = np.clip(new_nest, lower_bound, upper_bound)

            # Evaluate the new solution
            new_fitness = objective_function(new_nest)

            # If the new solution is better, replace the old solution
            if new_fitness < fitness[i]:
                nests[i] = new_nest
                fitness[i] = new_fitness

        # Abandon the worst nests
        for i in range(num_nests):
            if np.random.rand() < pa:  # Probability to abandon
                nests[i] = np.random.rand(dim) * (upper_bound - lower_bound) + lower_bound
                fitness[i] = objective_function(nests[i])

        # Find the current best nest
        best_nest_idx = np.argmin(fitness)
        best_nest = nests[best_nest_idx]
        best_fitness = fitness[best_nest_idx]

        # print(f"Iteration {iteration+1}, Best Fitness: {best_fitness}")

    return best_nest, best_fitness

# Example usage of Cuckoo Search

# Define the problem dimensions and bounds
dim = 5  # Dimension of the solution space
lower_bound = -5  # Lower bound of the search space
upper_bound = 5  # Upper bound of the search space

# Run Cuckoo Search
best_solution, best_fitness = cuckoo_search(objective_function, dim, lower_bound, upper_bound, num_nests=25, max_iter=100, pa=0.25)

print(f"Best Solution: {best_solution}")
print(f"Best Fitness: {best_fitness}")


Best Solution: [0.64982748 0.55961241 2.01501756 0.93987275 0.31984962]
Best Fitness: 5.78140211553397
